In [1]:
import numpy as np
from scipy.optimize import fsolve
import re
from numpyarray_to_latex import to_ltx
from IPython.display import display, Math

%load_ext autoreload

def disp(*mats, prefix="", sep=r"\ "):
    total_str = prefix
    for mat in mats:
        mat_obj = globals()[mat]
        ltx_str = to_ltx(mat_obj, fmt="{:.0f}")
        total_str = fr"{total_str}{sep}{mat}\ =\ {ltx_str}"

    display(Math(total_str))


def make_parity_and_order(n, k):
    order = [0] * n

    data_idx = 0
    parity_exp = 0
    parity_idx = k

    for i in range(n):
        if i+1 == 2 ** parity_exp:
            order[i] = parity_idx
            parity_idx += 1
            parity_exp += 1
        else:
            order[i] = data_idx
            data_idx += 1
    
    # Every number in [1, n] that is not a power of 2
    P = np.arange(1, n+1, dtype=np.uint8)
    P = P[np.log2(P) != np.floor(np.log2(P))]
    P = np.unpackbits(P.reshape((k, 1)), axis=1, count=r, bitorder="little")

    return P, order

In [2]:
# Information bits
k = 4

def parity_bits_fn(x):
    return (2 ** x) - x - k - 1

# Parity bits
x = fsolve(parity_bits_fn, x0=k).round(6).item()
r = int(np.ceil(x))

# Codeword length
n = k + r

raw_str = r"""
\begin{align}
2^r &\geq k+r+1  \\
f(x) &= 2^x-x-k-1 \\
r &= \lceil{x}\rceil \mid f(x) = 0,\ x > 0 \\
\end{align}
"""

raw_str = rf"""
{raw_str} \\
\qquad \\
k={k} \qquad r={r} \qquad n = k + r = {n}
"""
display(Math(raw_str))

<IPython.core.display.Math object>

In [3]:
# Identity matrices
I_k = np.eye(k)
I_n_k = np.eye(n-k)

# Parity equations
# P (k, n-k)
P, order = make_parity_and_order(n, k)

# Generator matrix (k, n)
G = np.hstack((I_k, P))

# Decoder matrix (k, n)
D = np.hstack((I_k, I_k * 0))

# Parity-check matrix (n-k, n)
H = np.hstack((P.T, I_n_k))
disp("G", "H", "D", prefix=r"\text{Initially}", sep=r"\\")

G = G[:, order]
H = H[:, order]
D = D[:, order]

G_t = G.T
disp("G_t", "H", "D", prefix=r"\text{After reordering}", sep=r"\\")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
# Input vector
_ = np.array([np.random.randint(0, 1 << k)], dtype=np.uint8)
# = np.array(ord('A'), dtype=np.uint8)
w_s = np.unpackbits(_, count=k, bitorder="little").reshape(k, 1)

# Codeword
c = G_t @ w_s % 2

disp("w_s", "c")

<IPython.core.display.Math object>

In [5]:
# Received codeword
R = np.copy(c).astype("u8")
modified_at = np.random.randint(0, n) + 1
R[modified_at-1] ^= 1

# Syndrome vector
z = H @ R % 2
disp("R", "z")

detected_at = np.packbits(z.astype("u8"), bitorder='little').item()
display(Math(rf"""
\text{{Error in Bit #{modified_at}}} \\
\text{{Syndrome detected error in Bit #{detected_at}}}
"""))

# Decoded vector
R[detected_at-1] ^= 1
w_d = D @ R

disp("w_d")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>